In [1]:
import tacco as tc
import scanpy as sp
import pandas as pd
import numpy as np
import anndata as ad

In [2]:
# Read the counts matrix
counts_matrix = pd.read_csv('MAPLE_Single_cell_counts_matrix.csv', index_col=0).T

# Read the metadata
metadata = pd.read_csv('MAPLE_Single_cell_metadata.csv', index_col=0)

# Create AnnData object
adata = ad.AnnData(X=counts_matrix)

# Add metadata to AnnData
adata.obs = metadata

# Verify the contents
adata

AnnData object with n_obs × n_vars = 21699 × 17415
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'Sample', 'Treatment', 'Injury', 'Status', 'Area', 'Type', 'Age', 'Age_Status', 'Sex', 'Treatment_Age_Status', 'Treatment_Age_Status_Sex', 'Treatment_Age_Status_Sex_Area', 'Names_Sample', 'Names_Age_Status', 'Names_Sample_Age_Status', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.2', 'seurat_clusters', 'predicted.id', 'prediction.score.Monocytes', 'prediction.score.AT2', 'prediction.score.Dendritics.cells', 'prediction.score.SCGB3A2.SCGB1A1..RAS', 'prediction.score.Alveolar.fibroblasts', 'prediction.score.CD4..cells', 'prediction.score.EC.Lymphatic', 'prediction.score.Alveolar.macrophage', 'prediction.score.Adventitial.fibroblasts', 'prediction.score.AT1', 'prediction.score.EC.Systemic', 'prediction.score.Mast.cells', 'prediction.score.NK', 'prediction.score.AT2.transitional', 'prediction.score.B.cells', 'prediction.score.Pericytes', 'prediction.score.Inflammatory

In [3]:
adata.obs['Sample'].value_counts()

PITTS2020_110_LL    11369
PITTS2020_110_UL     4353
WL_20166             3520
OSU10161_UL          1086
OSU10161_LL           753
WL_20132              618
Name: Sample, dtype: int64

In [4]:
adata.X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [5]:
labels=pd.read_csv('labels.csv',index_col=0)
adata.obs['Clusters']=labels

In [6]:
# Read the counts matrix
counts_matrix = pd.read_csv('MAPLE_Spatial_counts_matrix.csv', index_col=0).T

# Read the metadata
metadata = pd.read_csv('MAPLE_Spatial_cell_metadata.csv', index_col=0)

# Create AnnData object
adata1 = ad.AnnData(X=counts_matrix)

# Add metadata to AnnData
adata1.obs = metadata

# Verify the contents
adata1

AnnData object with n_obs × n_vars = 20373 × 18929
    obs: 'orig.ident', 'nCount_Spatial', 'nFeature_Spatial', 'Status', 'Area', 'Parenchymal_area', 'Sample', 'nCount_SCT', 'nFeature_SCT'

In [7]:
adata1

AnnData object with n_obs × n_vars = 20373 × 18929
    obs: 'orig.ident', 'nCount_Spatial', 'nFeature_Spatial', 'Status', 'Area', 'Parenchymal_area', 'Sample', 'nCount_SCT', 'nFeature_SCT'

In [8]:
adata1.X=adata1.X.astype('float64')

In [9]:
adata1.X

array([[ 2.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  6., 13.],
       [ 0.,  0.,  0., ...,  0.,  0.,  3.],
       [ 0.,  0.,  0., ...,  0.,  2., 15.]])

In [10]:
# using OSU10161_UL spatial + all scRNA-seq

In [11]:
adata1.obs['Sample'].value_counts()

OSU10161_LL         4334
OSU10161_UL         4205
OSU20132_PA         3174
PITTS2020_110_UL    3092
OSU20166_PA         2928
PITTS2020_110_LL    2640
Name: Sample, dtype: int64

In [12]:
adata_spatial=adata1[adata1.obs['Sample']=='OSU10161_UL'].copy()

In [13]:
tc.tl.annotate(adata_spatial, adata, annotation_key='Clusters', result_key='deconv_annotation')

Starting preprocessing
Annotation profiles were not found in `reference.varm["Clusters"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 11.14 seconds.
Starting annotation of data with shape (4205, 16020) and a reference of shape (21699, 16020) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  0.804425057336998 5.553481978627938
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 13.8 seconds.


AnnData object with n_obs × n_vars = 4205 × 18929
    obs: 'orig.ident', 'nCount_Spatial', 'nFeature_Spatial', 'Status', 'Area', 'Parenchymal_area', 'Sample', 'nCount_SCT', 'nFeature_SCT'
    obsm: 'deconv_annotation'
    varm: 'deconv_annotation'

In [14]:
adata_spatial.obsm['deconv_annotation'].to_csv('./maple_results/Atlas_OSU10161_UL_all.csv')

In [15]:
# using OSU10161_UL spatial + scRNA-seq OSU10161_UL, PITTS2020_110_UL

In [13]:
adata_spatial=adata1[adata1.obs['Sample']=='OSU10161_UL'].copy()

In [14]:
adata_sc=adata[(adata.obs['Sample']=='OSU10161_UL')|(adata.obs['Sample']=='PITTS2020_110_UL')].copy()

In [15]:
tc.tl.annotate(adata_spatial, adata_sc, annotation_key='Clusters', result_key='deconv_annotation')

Starting preprocessing
Annotation profiles were not found in `reference.varm["Clusters"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 3.69 seconds.
Starting annotation of data with shape (4205, 15410) and a reference of shape (5439, 15410) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  4.009996576059464 41.22731101321452
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 10.89 seconds.


AnnData object with n_obs × n_vars = 4205 × 18929
    obs: 'orig.ident', 'nCount_Spatial', 'nFeature_Spatial', 'Status', 'Area', 'Parenchymal_area', 'Sample', 'nCount_SCT', 'nFeature_SCT'
    obsm: 'deconv_annotation'
    varm: 'deconv_annotation'

In [16]:
adata_spatial.obsm['deconv_annotation'].to_csv('./maple_results/Atlas_OSU10161_UL_UL.csv')

In [17]:
# using PITTS2020_110_UL spatial + scRNA-seq all

In [18]:
adata_spatial=adata1[adata1.obs['Sample']=='PITTS2020_110_UL'].copy()

In [19]:
adata_sc=adata.copy()

In [20]:
tc.tl.annotate(adata_spatial, adata_sc, annotation_key='Clusters', result_key='deconv_annotation')

Starting preprocessing
Annotation profiles were not found in `reference.varm["Clusters"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 11.4 seconds.
Starting annotation of data with shape (3092, 16383) and a reference of shape (21699, 16383) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  1.593288482505194 9.860752444300232
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 11.89 seconds.


AnnData object with n_obs × n_vars = 3092 × 18929
    obs: 'orig.ident', 'nCount_Spatial', 'nFeature_Spatial', 'Status', 'Area', 'Parenchymal_area', 'Sample', 'nCount_SCT', 'nFeature_SCT'
    obsm: 'deconv_annotation'
    varm: 'deconv_annotation'

In [21]:
adata_spatial.obsm['deconv_annotation'].to_csv('./maple_results/Atlas_PITTS2020_110_UL_all.csv')

In [22]:
# using PITTS2020_110_UL spatial + OSU10161_UL, PITTS2020_110_UL

In [23]:
adata_spatial=adata1[adata1.obs['Sample']=='PITTS2020_110_UL'].copy()
adata_sc=adata[(adata.obs['Sample']=='OSU10161_UL')|(adata.obs['Sample']=='PITTS2020_110_UL')].copy()
tc.tl.annotate(adata_spatial, adata_sc, annotation_key='Clusters', result_key='deconv_annotation')
adata_spatial.obsm['deconv_annotation'].to_csv('./maple_results/Atlas_PITTS2020_110_UL_UL.csv')

Starting preprocessing
Annotation profiles were not found in `reference.varm["Clusters"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 4.0 seconds.
Starting annotation of data with shape (3092, 15624) and a reference of shape (5439, 15624) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  6.932909435344317 65.88955208831362
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 9.0 seconds.


In [24]:
# using OSU10161_LL spatial + all scRNA-seq

In [25]:
adata_spatial=adata1[adata1.obs['Sample']=='OSU10161_LL'].copy()
adata_sc=adata.copy()
tc.tl.annotate(adata_spatial, adata_sc, annotation_key='Clusters', result_key='deconv_annotation')
adata_spatial.obsm['deconv_annotation'].to_csv('./maple_results/Atlas_OSU10161_LL_all.csv')

Starting preprocessing
Annotation profiles were not found in `reference.varm["Clusters"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 11.25 seconds.
Starting annotation of data with shape (4334, 15575) and a reference of shape (21699, 15575) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  0.5303369841360026 4.759895226977724
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 14.26 seconds.


In [26]:
# using OSU10161_LL spatial +  scRNA-seq OSU10161_LL PITTS2020_110_LL

In [27]:
adata_spatial=adata1[adata1.obs['Sample']=='OSU10161_LL'].copy()
adata_sc=adata[(adata.obs['Sample']=='OSU10161_LL')|(adata.obs['Sample']=='PITTS2020_110_LL')].copy()
tc.tl.annotate(adata_spatial, adata_sc, annotation_key='Clusters', result_key='deconv_annotation')
adata_spatial.obsm['deconv_annotation'].to_csv('./maple_results/Atlas_OSU10161_LL_LL.csv')

Starting preprocessing
Annotation profiles were not found in `reference.varm["Clusters"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 6.69 seconds.
Starting annotation of data with shape (4334, 15383) and a reference of shape (12122, 15383) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  1.509721246965673 28.72895807723415
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 11.41 seconds.


In [28]:
# using PITTS2020_110_LL spatial +  scRNA-seq all

In [29]:
adata_spatial=adata1[adata1.obs['Sample']=='PITTS2020_110_LL'].copy()
adata_sc=adata.copy()
tc.tl.annotate(adata_spatial, adata_sc, annotation_key='Clusters', result_key='deconv_annotation')
adata_spatial.obsm['deconv_annotation'].to_csv('./maple_results/Atlas_PITTS2020_110_LL_all.csv')

Starting preprocessing
Annotation profiles were not found in `reference.varm["Clusters"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 10.95 seconds.
Starting annotation of data with shape (2640, 16350) and a reference of shape (21699, 16350) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  0.7430872730949775 5.434584503771209
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 10.0 seconds.


In [30]:
# using PITTS2020_110_LL spatial +  scRNA-seq OSU10161_LL PITTS2020_110_LL

In [31]:
adata_spatial=adata1[adata1.obs['Sample']=='PITTS2020_110_LL'].copy()
adata_sc=adata[(adata.obs['Sample']=='OSU10161_LL')|(adata.obs['Sample']=='PITTS2020_110_LL')].copy()
tc.tl.annotate(adata_spatial, adata_sc, annotation_key='Clusters', result_key='deconv_annotation')
adata_spatial.obsm['deconv_annotation'].to_csv('./maple_results/Atlas_PITTS2020_110_LL_LL.csv')

Starting preprocessing
Annotation profiles were not found in `reference.varm["Clusters"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 6.21 seconds.
Starting annotation of data with shape (2640, 16019) and a reference of shape (12122, 16019) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  1.6233233061530103 30.52205463268122
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 8.14 seconds.


In [32]:
# using OSU20132_PA spatial +  scRNA-seq all

In [33]:
adata_spatial=adata1[adata1.obs['Sample']=='OSU20132_PA'].copy()
adata_sc=adata.copy()
tc.tl.annotate(adata_spatial, adata_sc, annotation_key='Clusters', result_key='deconv_annotation')
adata_spatial.obsm['deconv_annotation'].to_csv('./maple_results/Atlas_OSU20132_PA_all.csv')

Starting preprocessing
Annotation profiles were not found in `reference.varm["Clusters"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 10.64 seconds.
Starting annotation of data with shape (3174, 15607) and a reference of shape (21699, 15607) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  0.7705888469070125 5.394154418055931
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 10.6 seconds.


In [34]:
# using OSU20132_PA spatial +  scRNA-seq WL_20132,WL_20166

In [35]:
adata_spatial=adata1[adata1.obs['Sample']=='OSU20132_PA'].copy()
adata_sc=adata[(adata.obs['Sample']=='WL_20132')|(adata.obs['Sample']=='WL_20166')].copy()
tc.tl.annotate(adata_spatial, adata_sc, annotation_key='Clusters', result_key='deconv_annotation')
adata_spatial.obsm['deconv_annotation'].to_csv('./maple_results/Atlas_OSU20132_PA_PA.csv')

Starting preprocessing
Annotation profiles were not found in `reference.varm["Clusters"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 2.65 seconds.
Starting annotation of data with shape (3174, 14575) and a reference of shape (4138, 14575) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  2.838390352407359 14.505972399475377
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 7.58 seconds.


In [36]:
# using OSU20166_PA spatial +  scRNA-seq all

In [37]:
adata_spatial=adata1[adata1.obs['Sample']=='OSU20166_PA'].copy()
adata_sc=adata.copy()
tc.tl.annotate(adata_spatial, adata_sc, annotation_key='Clusters', result_key='deconv_annotation')
adata_spatial.obsm['deconv_annotation'].to_csv('./maple_results/Atlas_OSU20166_PA_all.csv')

Starting preprocessing
Annotation profiles were not found in `reference.varm["Clusters"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 11.4 seconds.
Starting annotation of data with shape (2928, 17229) and a reference of shape (21699, 17229) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  1.307511560884461 2.123197613143025
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 11.38 seconds.


In [38]:
# using OSU20166_PA spatial +  scRNA-seq WL_20132,WL_20166

In [39]:
adata_spatial=adata1[adata1.obs['Sample']=='OSU20166_PA'].copy()
adata_sc=adata[(adata.obs['Sample']=='WL_20132')|(adata.obs['Sample']=='WL_20166')].copy()
tc.tl.annotate(adata_spatial, adata_sc, annotation_key='Clusters', result_key='deconv_annotation')
adata_spatial.obsm['deconv_annotation'].to_csv('./maple_results/Atlas_OSU20166_PA_PA.csv')

Starting preprocessing
Annotation profiles were not found in `reference.varm["Clusters"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 2.57 seconds.
Starting annotation of data with shape (2928, 15304) and a reference of shape (4138, 15304) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  3.968230286080189 9.031180026359946
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 7.13 seconds.


In [40]:
# using whole data

In [32]:
tc.tl.annotate(adata1, adata, annotation_key='Clusters', result_key='deconv_annotation')

Starting preprocessing
Finished preprocessing in 14.85 seconds.
Starting annotation of data with shape (20373, 17234) and a reference of shape (21699, 17234) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  5.451601885029508 29.223875816905124
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 45.72 seconds.


AnnData object with n_obs × n_vars = 20373 × 18929
    obs: 'orig.ident', 'nCount_Spatial', 'nFeature_Spatial', 'Status', 'Area', 'Parenchymal_area', 'Sample', 'nCount_SCT', 'nFeature_SCT'
    obsm: 'deconv_annotation'
    varm: 'deconv_annotation'

In [35]:
adata1.obsm['deconv_annotation'].to_csv('Manual_whole_deconv.csv')

In [37]:
adata.obs['Status'].value_counts()

IPF        17561
Healthy     4138
Name: Status, dtype: int64

In [ ]:
# only for healthy data

In [8]:
scrna_subset=adata[adata.obs['Status']=='Healthy'].copy()
spatial_subset=adata1[adata1.obs['Status']=='Healthy'].copy()

In [10]:
spatial_subset

AnnData object with n_obs × n_vars = 6102 × 18929
    obs: 'orig.ident', 'nCount_Spatial', 'nFeature_Spatial', 'Status', 'Area', 'Parenchymal_area', 'Sample', 'nCount_SCT', 'nFeature_SCT'

In [11]:
tc.tl.annotate(spatial_subset, scrna_subset, annotation_key='Clusters', result_key='deconv_annotation')

Starting preprocessing
Annotation profiles were not found in `reference.varm["Clusters"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 3.64 seconds.
Starting annotation of data with shape (6102, 15305) and a reference of shape (4138, 15305) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  6.670979136524565 19.675134702947602
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 12.86 seconds.


AnnData object with n_obs × n_vars = 6102 × 18929
    obs: 'orig.ident', 'nCount_Spatial', 'nFeature_Spatial', 'Status', 'Area', 'Parenchymal_area', 'Sample', 'nCount_SCT', 'nFeature_SCT'
    obsm: 'deconv_annotation'
    varm: 'deconv_annotation'

In [12]:
spatial_subset.obsm['deconv_annotation'].to_csv('Manual_Healthy_deconv.csv')

In [13]:
# using IPF UL

In [22]:
scrna_subset=adata[(adata.obs['Status']=='IPF')&(adata.obs['Area']=='Upper Lobe')].copy()
spatial_subset=adata1[(adata1.obs['Status']=='IPF')&(adata1.obs['Parenchymal_area']=='UL')].copy()

In [23]:
tc.tl.annotate(spatial_subset, scrna_subset, annotation_key='Clusters', result_key='deconv_annotation')

Starting preprocessing
Annotation profiles were not found in `reference.varm["Clusters"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 4.49 seconds.
Starting annotation of data with shape (7297, 15627) and a reference of shape (5439, 15627) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  10.885891358219489 106.23804704890262
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 15.45 seconds.


AnnData object with n_obs × n_vars = 7297 × 18929
    obs: 'orig.ident', 'nCount_Spatial', 'nFeature_Spatial', 'Status', 'Area', 'Parenchymal_area', 'Sample', 'nCount_SCT', 'nFeature_SCT'
    obsm: 'deconv_annotation'
    varm: 'deconv_annotation'

In [24]:
spatial_subset.obsm['deconv_annotation'].to_csv('Manual_IPF_UL_deconv.csv')

In [25]:
# using IPF LL

In [26]:
scrna_subset=adata[(adata.obs['Status']=='IPF')&(adata.obs['Area']=='Lower Lobe')].copy()
spatial_subset=adata1[(adata1.obs['Status']=='IPF')&(adata1.obs['Parenchymal_area']=='LL')].copy()

In [27]:
tc.tl.annotate(spatial_subset, scrna_subset, annotation_key='Clusters', result_key='deconv_annotation')

Starting preprocessing
Annotation profiles were not found in `reference.varm["Clusters"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 7.25 seconds.
Starting annotation of data with shape (6974, 16044) and a reference of shape (12122, 16044) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Clusters, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  3.0683157556306417 58.478898781943144
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 15.19 seconds.


AnnData object with n_obs × n_vars = 6974 × 18929
    obs: 'orig.ident', 'nCount_Spatial', 'nFeature_Spatial', 'Status', 'Area', 'Parenchymal_area', 'Sample', 'nCount_SCT', 'nFeature_SCT'
    obsm: 'deconv_annotation'
    varm: 'deconv_annotation'

In [28]:
spatial_subset.obsm['deconv_annotation'].to_csv('Manual_IPF_LL_deconv.csv')

In [31]:
1024*19

19456